https://www.frontier.maxell.co.jp/blog/posts/42.html

画像の色抽出 ～RGBとHSVでの色抽出～
https://www.frontier.maxell.co.jp/blog/posts/43.html

In [40]:
!pip install open3d

import sys
import numpy as np
import pandas as pd
import open3d as o3d

o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
# o3d.core.set_use_openmp(False)
# o3d.core.set_use_cuda(True) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
# Open3DのCUDA対応のバイナリは提供されていないため自分でbuildする必要がある。
#https://zenn.dev/yutashx/articles/open3d_build

import torch

# テンソルを作成してGPU上に置く
x = torch.zeros((3, 3)).cuda()

# デバイス名を表示する
print(x.device)

cuda:0


In [42]:
!pip install trimesh
import trimesh
import numpy as np
from shapely.geometry import LineString
%pylab inline
%config InlineBackend.figure_format = 'svg'
# http://3dcad.garyoutensei.com/6-2_CloudCompare.html#CC7
# https://stackoverflow.com/questions/60038105/find-what-is-material-and-what-is-air-in-2d-cross-sections-from-3d-stl-files-us


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/pylab.py:159: UserWarning:

pylab import has clobbered these variables: ['indices', 'cm']
`%matplotlib` prevents importing * from pylab and numpy



In [43]:
from sklearn.cluster import KMeans

In [44]:
from scipy.spatial.distance import pdist, squareform

In [45]:
def down_sample_pcd_plotly(pcd):
  pcd_down = pcd.uniform_down_sample(every_k_points=10)
  o3d.visualization.draw_plotly([pcd_down])

def down_sample_colorbar_pcd_plotly(pcd):
  color_b = o3d.geometry.PointCloud()
  color_b.points = o3d.utility.Vector3dVector(pcd.colors)
  color_b.colors = o3d.utility.Vector3dVector(pcd.colors)
  down_sample_pcd_plotly(color_b)

def colorbar_pcd_plotly(pcd):
  color_b = o3d.geometry.PointCloud()
  color_b.points = o3d.utility.Vector3dVector(pcd.colors)
  color_b.colors = o3d.utility.Vector3dVector(pcd.colors)
  o3d.visualization.draw_plotly(color_b)


In [46]:
filename = '/content/drive/MyDrive/PointCloudData/circle_curl.ply'
pcd_doki = o3d.io.read_point_cloud(filename)

## 現物とカラーチャートをダウンサンプリングして表示
down_sample_pcd_plotly(pcd_doki)
down_sample_colorbar_pcd_plotly(pcd_doki)

Output hidden; open in https://colab.research.google.com to view.

In [47]:
## 座標とRGBを配列化
xyz = ['x','y','z']
rgb = ['r','g','b']
rgb2 = ['r2','g2','b2']

def pcd_to_df(pcd):
  xyz = ['x','y','z']
  rgb = ['r','g','b']

  np_doki = np.asarray(pcd.points)
  np_col = np.asarray(pcd.colors) #* 255.0
  ## xyzrgb　の並びにする
  df1 = pd.DataFrame(data=np_doki, columns=xyz)
  df2 = pd.DataFrame(np_col, columns=rgb)
  df_pcd = pd.concat([df1, df2], axis=1)
  return df_pcd

### dfを点群として表示
def down_sample_df_plotly(df):
  xyz = ['x','y','z']
  rgb = ['r','g','b']

  pcd_doki2 = o3d.geometry.PointCloud()
  pcd_doki2.points = o3d.utility.Vector3dVector(df[xyz].values)
  pcd_doki2.colors = o3d.utility.Vector3dVector(df[rgb].values)
  down_sample_pcd_plotly(pcd_doki2)

def df_plotly(df):
  xyz = ['x','y','z']
  rgb = ['r','g','b']

  pcd_doki2 = o3d.geometry.PointCloud()
  pcd_doki2.points = o3d.utility.Vector3dVector(df[xyz].values)
  pcd_doki2.colors = o3d.utility.Vector3dVector(df[rgb].values)
  # down_sample_pcd_plotly(pcd_doki2)
  o3d.visualization.draw_plotly([pcd_doki2])

def colorbar_df_plotly(df):
  xyz = ['x','y','z']
  rgb = ['r','g','b']

  pcd_doki2 = o3d.geometry.PointCloud()
  pcd_doki2.points = o3d.utility.Vector3dVector(df[rgb].values)
  pcd_doki2.colors = o3d.utility.Vector3dVector(df[rgb].values)
  o3d.visualization.draw_plotly([pcd_doki2])
  # down_sample_pcd_plotly(pcd_doki2)

def separate_df_plotly(df_xyz,df_rgb):

  pcd_doki2 = o3d.geometry.PointCloud()
  pcd_doki2.points = o3d.utility.Vector3dVector(df_xyz.values)
  pcd_doki2.colors = o3d.utility.Vector3dVector(df_rgb.values)
  down_sample_pcd_plotly(pcd_doki2)
  # o3d.visualization.draw_plotly([pcd_doki2])



In [48]:
df_pcd = pcd_to_df(pcd_doki)
df_pcd['border'] = df_pcd['r']+0.1
df_pcd['green_border'] = df_pcd['b']+0.1
df_pcd['type'] = 'normal'
df_pcd2 = df_pcd.copy()

# 緑
green_rows = df_pcd.query('g > green_border & g >= r & g > b & g >0.2')
green_rows['type'] = 'green' 

# 黄色：口縁
yellow_rows = df_pcd.query('r > 0.6 & g > 0.5 & b < 0.4')
yellow_rows['type'] = 'yellow' 

# 青：底部
blue_rows = df_pcd.query('b > border & b > g & b >0.1')
blue_rows['type'] = 'blue'

## 元dataframeを更新
df_pcd2.update(green_rows)
df_pcd2.update(blue_rows)

print(green_rows)
# 表示
df_plotly(green_rows)
colorbar_df_plotly(green_rows)

<ipython-input-48-ea8473c47bc4>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-48-ea8473c47bc4>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



               x         y         z         r         g         b    border  \
224451  0.516757 -0.117714 -0.207538  0.541176  0.615686  0.490196  0.641176   
224464  0.517788 -0.115455 -0.207523  0.556863  0.627451  0.470588  0.656863   
224466  0.517689 -0.115839 -0.207525  0.533333  0.615686  0.447059  0.633333   
224467  0.516772 -0.114767 -0.207428  0.580392  0.662745  0.494118  0.680392   
224479  0.516167 -0.116333 -0.207501  0.533333  0.619608  0.435294  0.633333   
...          ...       ...       ...       ...       ...       ...       ...   
914283  0.110813 -0.489948 -0.222098  0.537255  0.627451  0.415686  0.637255   
914355  0.110534 -0.491792 -0.222015  0.431373  0.486275  0.384314  0.531373   
915224  0.112633 -0.487959 -0.221908  0.454902  0.513725  0.400000  0.554902   
915226  0.112570 -0.488301 -0.221943  0.501961  0.564706  0.423529  0.601961   
918246  0.113260 -0.488670 -0.221891  0.450980  0.509804  0.396078  0.550980   

        green_border   type  
224451   

In [49]:
df_plotly(blue_rows)
colorbar_df_plotly(blue_rows)

Output hidden; open in https://colab.research.google.com to view.

In [50]:
# 底部の中間の高さを取得する（ある程度点群が水平に取得されている前提→先に平面を探す）
max_val = blue_rows['z'].max()
min_val = blue_rows['z'].min()
border_val = np.mean([max_val,min_val])/2
# print(max_val,min_val)
green_rows['z_border'] = border_val

# topとbottom（scale用） に区別して元のDFに書き戻す
top_rows = green_rows.query('z > z_border')
top_rows['type'] = 'green_top' 
scale_rows = green_rows.query('z <= z_border')
scale_rows['type'] = 'green_scale' 
df_pcd2.update(top_rows)
df_pcd2.update(scale_rows)

<ipython-input-50-052f9d4836c7>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-50-052f9d4836c7>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-50-052f9d4836c7>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [51]:
df_plotly(top_rows)
df_plotly(scale_rows)

In [52]:
def dog_axis(right_eye, left_eye, nose ,zzz):
    origin = (right_eye + left_eye) / 2
    #原点を両目の中点とする
    px = right_eye - left_eye
    # x軸は左目→右目
    py = nose - origin
    # 「しっぽから鼻にむかう直線」

    """それぞれの単位ベクトルを計算"""
    ex = px / np.linalg.norm(px)
    ez = np.cross(px, py) / np.linalg.norm(np.cross(px, py))
    ey = np.cross(ez, ex)

    """変換行列 4x4 を作成"""
    #ローカル座標（犬から見た座標）を絶対座標に変換する行列
    R_LtoG = np.identity(4)

    R_LtoG[:3, :3] = np.array([ex, ey, ez]).T
    #回転行列を代入
    R_LtoG[:3, 3] = origin
    #移動ベクトルを代入

    #絶対座標からローカル座標（犬からみた座標）に変換する行列
    R_GtoL = np.identity(4)*zzz
    R_GtoL[:3, :3] = np.array([ex, ey, ez])
    R_GtoL[:3, 3] = - np.dot(np.array([ex, ey, ez]), origin)

    return (R_GtoL)

def get_R_GtoL(df):
  # 3点を抽出
  X = df[xyz].values
  kmeans = KMeans(n_clusters=3, random_state=9).fit(X)
  points = kmeans.cluster_centers_

  # 2点間の距離を計算
  distances = pdist(points)

  # 距離行列を作成
  distance_matrix = squareform(distances)

  # 最も距離が大きい2点を探す
  i, j = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)

  # 残りの1点のインデックスを取得
  k = np.setdiff1d(np.arange(3), [i, j])[0]
  point_i = points[i]
  point_j = points[j]
  point_k = points[k]

  R_GtoL = dog_axis(point_i, point_j, point_k,1)# あとでより点が少なくなる側を＋とするように変換しなくてはいけない
  return R_GtoL

In [53]:
# 縮尺用の点を元に台座部分を識別
# 水平に位置移動
R_GtoL= get_R_GtoL(scale_rows)
pcd_leveling = pcd_doki.transform(R_GtoL)
## ダウンサンプリングして表示
down_sample_pcd_plotly(pcd_leveling)

# # 変換後の座標をデータフレームに格納
df_leveling = pcd_to_df(pcd_leveling)
df_leveling['type'] = df_pcd2['type']
# df_pcd3['levelingx'] = df_leveling['x']
# df_pcd3['levelingy'] = df_leveling['y']
# df_pcd3['levelingz'] = df_leveling['z']
df_leveling['levelingz_border'] = df_leveling[df_leveling['type'] == 'green_scale']['z'].max()

bottom_rows = df_leveling.query('z < levelingz_border')
# print(bottom_rows[xyz])

bottom_rows['type'] = 'bottom' 
# df_pcd2.update(bottom_rows)
# print(bottom_rows[xyz])

# down_sample_df_plotly(bottom_rows)

Output hidden; open in https://colab.research.google.com to view.

In [54]:

# 平面検出のためのパラメータ設定
plane_distance_threshold = 0.01
ransac_n = 100
num_iterations = 5000

# RANSAC平面検出を実行
plane_model, inliers = pcd_doki.segment_plane(distance_threshold=plane_distance_threshold,
                                          ransac_n=ransac_n,
                                          num_iterations=num_iterations)


# 平面に属する点だけの点群を作成
pcd_plane = pcd_doki.select_by_index(inliers)

# 平面以外の点だけの点群を作成
pcd_nonplane = pcd_doki.select_by_index(inliers, invert=True)

# 検出された平面を可視化
down_sample_pcd_plotly(pcd_plane)
down_sample_pcd_plotly(pcd_nonplane)




Output hidden; open in https://colab.research.google.com to view.

In [59]:
# DBSCANアルゴリズムによるクラスタリング
# pcd_down = pcd_nonplane.uniform_down_sample(every_k_points=2)

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        pcd_nonplane.cluster_dbscan(eps=0.02, min_points=50, print_progress=True))

# クラスタごとに異なる色を設定
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")
colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0

# 各点に色を割り当てて表示
pcd_down_color = o3d.geometry.PointCloud(pcd_nonplane)
pcd_down_color.colors = o3d.utility.Vector3dVector(colors[:, :3])
down_sample_pcd_plotly(pcd_down_color)


Output hidden; open in https://colab.research.google.com to view.

In [82]:
pcd_list = []
for i in range(max_label+1):
    indices = np.where(labels == i)[0]
    pcd_cluster = pcd_down_color.select_by_index(indices)
    pcd_cluster.paint_uniform_color(colors[i][:3])
    pcd_list.append(pcd_cluster)

# 各クラスタを表示
down_sample_pcd_plotly(pcd_list[0])

Output hidden; open in https://colab.research.google.com to view.

In [85]:
df_nonplane = pcd_to_df(pcd_nonplane)
df_nonplane['border'] = df_nonplane['r']+0.1
df_nonplane['green_border'] = df_nonplane['b']+0.1
df_nonplane['label'] = labels
# print(df_nonplane)

# # 緑
# green_rows = df_pcd.query('g > green_border & g >= r & g > b & g >0.2')
# green_rows['type'] = 'green' 

# # 黄色：口縁
# yellow_rows = df_pcd.query('r > 0.6 & g > 0.5 & b < 0.4')
# yellow_rows['type'] = 'yellow' 

# # 青：底部
blue_rows = df_nonplane.query('b > border & b > g & b >0.1')
print(blue_rows)
# blue_rows['color'] = 'blue'

# ## 元dataframeを更新
# df_pcd2.update(green_rows)
# df_pcd2.update(blue_rows)

# print(green_rows)
# # 表示
df_plotly(blue_rows)
# colorbar_df_plotly(green_rows)

Output hidden; open in https://colab.research.google.com to view.

In [204]:
def max_distance(df):
    """
    Calculates the maximum distance between any two points in a dataframe containing
    3D point coordinates.
    
    Parameters:
    df (pandas.DataFrame): A dataframe containing the 3D point coordinates. The dataframe
                           must have columns named 'x', 'y', and 'z'.
                           
    Returns:
    float: The maximum distance between any two points in the dataframe.
    """
    # Extract the coordinates as a numpy array
    coords = df[['x', 'y', 'z']].values
    
    # Calculate the pairwise distance matrix between all points
    dist_matrix = np.sqrt(((coords[:, None] - coords)**2).sum(-1))
    
    # Ignore the diagonal (distance between a point and itself)
    np.fill_diagonal(dist_matrix, 0)
    
    # Return the maximum distance
    max_distance = dist_matrix.max()

    return max_distance

max_d = max_distance(scale_rows[xyz])
scale = 1.72/max_d
print(scale)

2.186150496952237


In [205]:
# 縮尺変更と表示
df_pcd['scalex'] = df_pcd['x'] * scale
df_pcd['scaley'] = df_pcd['y'] * scale
df_pcd['scalez'] = df_pcd['z'] * scale


df_pcd_normal = df_pcd[df_pcd['type'] == 'normal']
down_sample_df_plotly(df_pcd_normal)

Output hidden; open in https://colab.research.google.com to view.

In [206]:
# 口縁に対して水平に変換
# 3点を抽出
# X = green_rows[xyz].values
# kmeans = KMeans(n_clusters=3, random_state=9).fit(X)
#  #green_rows['result'] = kmeans.labels_ 
# #df_centers = pd.DataFrame(data=kmeans.cluster_centers_, columns=xyz)
# print(kmeans.cluster_centers_)

In [207]:
# 元の点群から青と緑を引いて位置移動
# df_pcd_normal = df_pcd[df_pcd['type'] == 'normal']

# pcd_normal = o3d.geometry.PointCloud()
# pcd_normal.points = o3d.utility.Vector3dVector(df_pcd_normal[xyz].values)
# pcd_normal.colors = o3d.utility.Vector3dVector(df_pcd_normal[rgb].values)

pcd_moved = pcd_normal.transform(get_R_GtoL(top_rows))


## ダウンサンプリングして表示
down_sample_pcd_plotly(pcd_moved)

Output hidden; open in https://colab.research.google.com to view.

In [208]:
def calculate_circle(points):
    """
    3つの点を通る円の中心座標と半径を計算する関数

    Parameters
    ----------
    points : array-like, shape (3, 2)
        3つのXY座標を持つNumPy配列

    Returns
    -------
    float, float, float
        円の中心座標のX座標、Y座標、半径
    """

    # 各点の座標を変数に代入
    x1, y1 = points[0]
    x2, y2 = points[1]
    x3, y3 = points[2]

    # 3点を通る円の中心座標を計算
    D = 2 * (x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
    Ux = ((x1 ** 2 + y1 ** 2) * (y2 - y3) + (x2 ** 2 + y2 ** 2) * (y3 - y1) + (x3 ** 2 + y3 ** 2) * (y1 - y2)) / D
    Uy = ((x1 ** 2 + y1 ** 2) * (x3 - x2) + (x2 ** 2 + y2 ** 2) * (x1 - x3) + (x3 ** 2 + y3 ** 2) * (x2 - x1)) / D

    # 3点を通る円の半径を計算
    R = np.sqrt((x1 - Ux) ** 2 + (y1 - Uy) ** 2)

    return Ux, Uy, R

In [209]:
pcd_z = o3d.geometry.PointCloud()
pcd_z.points = o3d.utility.Vector3dVector(points)
pcd_z_moved = pcd_z.transform(R_GtoL)
np_z = np.asarray(pcd_z_moved.points)

# np_z[:, 2] = 0
# print(np_z)

# center,radius = calculate_circle_radius(np_z)
# print(center,radius)

cx, cy, radius = calculate_circle(np_z[:, :2])

# 角度を0から360度まで0.1刻みで生成
theta = np.linspace(0, 2 * np.pi, 100)

# 円のx, y座標を計算
x = radius * np.cos(theta) + cx
y = radius * np.sin(theta) + cy

# 円を描画
plt.plot(x, y, 'b-', linewidth=1)  # 線の色を青、線幅を2に設定

# X座標を取得
xx = np_z[:, 0]
# Y座標を取得
yy = np_z[:, 1]
plt.plot(xx,yy , 'ro')  # 赤色の点をプロット

plt.axis('equal')  # X軸とY軸のスケールを等しく設定
plt.xlabel('X')  # X軸のラベルを設定
plt.ylabel('Y')  # Y軸のラベルを設定

print([cx,cy])
print(radius)
plt.show()  # グラフを表示

[-0.011721901611795432, -0.21653405629991573]
0.00035219008184686316


In [210]:
def move_points_to_origin(points, origin):
    """
    点群データを指定された原点に移動する関数

    Parameters
    ----------
    points : numpy.ndarray, shape (N, 3)
        3次元の点群データを表すnumpy配列
    origin : numpy.ndarray, shape (3,)
        移動先の原点としたい任意の点の座標を表すnumpy配列

    Returns
    -------
    moved_points : numpy.ndarray, shape (N, 3)
        移動後の点群データを表すnumpy配列
    """
    # 移動前の点群データの重心を計算
    centroid = np.mean(points, axis=0)

    # 移動先の原点としたい任意の点と重心との差を計算
    translation = origin - centroid

    # 点群データを移動
    moved_points = points + translation

    return moved_points


In [ ]:
translation = np.array([0,0,0]) - np.array([0,cy,0])
moved_points = np.asarray(pcd_moved.points) - np.array([0,cy,0])


pcd_moved_points = o3d.geometry.PointCloud()
pcd_moved_points.points = o3d.utility.Vector3dVector(moved_points)
pcd_moved_points.colors = o3d.utility.Vector3dVector(pcd_moved.colors)

## ダウンサンプリングして表示
pcd_doki_down3 = pcd_moved_points.uniform_down_sample(every_k_points=10)
o3d.visualization.draw_plotly([pcd_doki_down3])

In [ ]:
# メッシュ化

# 法線推定
pcd_doki_down3.estimate_normals()

# 法線推定調整時の半径が重要！土器の最薄の厚みを考えないと
pcd_doki_down3.orient_normals_consistent_tangent_plane(10)

# ポアソン曲面
with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    mesh_doki, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd_doki_down3, depth=10)
#ダウンサンプリングしたものしか処理できないけど計算資源あればいけるかも

o3d.visualization.draw_plotly([mesh_doki])

In [ ]:
import matplotlib.pyplot as plt
def path2Dshow(Path2D,filename):
        if Path2D.is_closed:
              plt.cla()
              axis = plt.gca()
              axis.set_aspect('equal', 'datalim')

              for i, points in enumerate(Path2D.discrete):
                  # 転置が必要
                  # axis.plot(*points.T, color=(0,0,1))
                  axis.plot(*points[:, ::-1].T, color=(0,0,1), alpha = 0.4)
              
              plt.hlines(0,-radius,radius,color="blue", alpha = 0.4)
              plt.vlines(0,-radius,0,color="blue", alpha = 0.4)

              # 目盛線を設定
              axis.grid(which = "major", axis = "x", color = "green", alpha = 0.3, linewidth = 1)
              axis.grid(which = "major", axis = "y", color = "green", alpha = 0.3, linewidth = 1)
              
              # 補助目盛線を設定
              axis.minorticks_on()
              axis.grid(which = "minor", axis = "x", color = "green", alpha = 0.3, linestyle = "--", linewidth = 0.5)
              axis.grid(which = "minor", axis = "y", color = "green", alpha = 0.3, linestyle = "--", linewidth = 0.5)
              
              #plt.savefig(filename, format="png", dpi=300)
              plt.show()
              
              return axis
        else:
            return 'is_not_closed'

In [ ]:
# open3d.cpu.pybind.geometry.TriangleMesh を trimesh.base.Trimesh オブジェクトに変換
trimesh_doki = trimesh.Trimesh(vertices=np.asarray(mesh_doki.vertices),
                               faces=np.asarray(mesh_doki.triangles))

slice = trimesh_doki.section(plane_origin=(0,0,0),#mesh2.centroid,
                     plane_normal=[1,0,0])

to_2D = trimesh.geometry.align_vectors([0,0,1], [1,0,0])

slice_2D, to_3D = slice.to_planar(to_2D = to_2D)
# slice_2D, to_3D = slice.to_planar()

# print(slice_2D)
filename = '/content/drive/MyDrive/PointCloudData/cross2.png'
# path2Dshow(slice_2D,filename)

In [ ]:
path2Dshow(slice_2D,filename)

print(radius)

In [ ]:
# def find_circle_center(p1, p2, p3):
#     # 3つの点の座標を取得
#     x1, y1, z1 = p1
#     x2, y2, z2 = p2
#     x3, y3, z3 = p3

#     # 3つの点を含む平面を定義
#     normal = np.cross(p2 - p1, p3 - p1)
    
#     # 平面の法線ベクトルを正規化
#     normal = normal / np.linalg.norm(normal)

#     # 3つの点の中点を計算
#     mid = (p1 + p2 + p3) / 3

#     # 中点から平面の法線ベクトルに垂直なベクトルを求める
#     perpendicular = np.cross(mid - p1, normal)

#     # 円の中心を計算
#     center = mid + perpendicular

#     # 各点から中心座標への距離を計算し、最大値を円周の半径とする
#     distances = np.linalg.norm(points - center, axis=1)
#     radius = np.max(distances)

#     return center,radius

In [ ]:
# def calculate_circle_radius(points):
#     """
#     ３点の３次元座標を使って、それを通る円周の半径を計算する関数

#     Parameters
#     ----------
#     points : numpy array, shape (3, 3)
#         ３点の３次元座標

#     Returns
#     -------
#     float
#         円周の半径
#     """
#     # 平面の法線ベクトルを求める
#     v1 = points[1] - points[0]
#     v2 = points[2] - points[0]
#     normal = np.cross(v1, v2)

#     # ３点の中心座標を求める
#     center = np.mean(points, axis=0)

#     # 各点から中心座標への距離を計算し、最大値を円周の半径とする
#     distances = np.linalg.norm(points - center, axis=1)
#     radius = np.max(distances)

#     return center,radius

In [ ]:
# def generate_circle_point_cloud(points):
#     """
#     ３点の３次元座標を使って、それを通る同一平面上の円の点群を生成する関数

#     Parameters
#     ----------
#     points : numpy array, shape (3, 3)
#         ３点の３次元座標

#     Returns
#     -------
#     o3d.geometry.PointCloud
#         生成された円の点群
#     """
#     # 3点を通る平面の法線ベクトルを計算
#     v1 = points[1] - points[0]
#     v2 = points[2] - points[0]
#     normal = np.cross(v1, v2)

#     # 3点が同一平面上にあるかを判定
#     if np.linalg.norm(normal) == 0:
#         raise ValueError("Input points are not on the same plane.")

#     # 平面の方程式 Ax + By + Cz + D = 0 の係数を計算
#     A, B, C = normal
#     D = -np.dot(normal, points[0])

#     # 平面に垂直なベクトルを計算
#     if A != 0:
#         v = np.array([(-D-B)/A, 1, 0])
#     elif B != 0:
#         v = np.array([1, (-D-A)/B, 0])
#     else:
#         v = np.array([1, 0, (-D-A)/C])

#     # 平面に垂直なベクトルと法線ベクトルの外積を計算し、円の法線ベクトルを得る
#     circle_normal = np.cross(v, normal)

#     # 3点を通る円の中心座標を計算
#     center = np.mean(points, axis=0)

#     # 円上の点を生成
#     theta = np.linspace(0, 2 * np.pi, 100)
#     x = center[0] + np.cos(theta) * circle_normal[0]
#     y = center[1] + np.cos(theta) * circle_normal[1]
#     z = center[2] + np.cos(theta) * circle_normal[2]
#     points = np.vstack((x, y, z)).T

#     # 点群を生成
#     point_cloud = o3d.geometry.PointCloud()
#     point_cloud.points = o3d.utility.Vector3dVector(points)

#     return point_cloud

In [ ]:
# def generate_sphere_point_cloud(points):
#     """
#     ３点を表面に持つ球の点群を生成する関数

#     Parameters
#     ----------
#     points : numpy array, shape (3, 3)
#         ３点の３次元座標

#     Returns
#     -------
#     o3d.geometry.PointCloud
#         生成された球の点群
#     """
#     # 3点の座標を取得
#     point1 = points[0]
#     point2 = points[1]
#     point3 = points[2]

#     # 3点を通る球の中心座標を計算
#     # 3点を結ぶベクトルとその垂直二つを計算
#     v1 = point2 - point1
#     v2 = point3 - point1
#     normal = np.cross(v1, v2)
#     d = -np.dot(normal, point1)
#     center = -normal * d

#     # 3点間の距離を計算して最大のものを半径とする
#     radius = max(np.linalg.norm(point1 - center),
#                 np.linalg.norm(point2 - center),
#                 np.linalg.norm(point3 - center))

#     # 球の表面の点群を生成
#     sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
#     sphere.translate(center)

#     # 3点を球の外側に持つように球の中心座標を調整
#     for i in range(3):
#         if np.dot(sphere.vertices[i], normal) > d:
#             sphere.vertices[i] = (sphere.vertices[i] - center) * radius / (radius + 1) + center

#     # 球の点群を点群データに変換
#     point_cloud = o3d.geometry.PointCloud()
#     point_cloud.points = o3d.utility.Vector3dVector(np.asarray(sphere.vertices))

#     return point_cloud

In [ ]:
# def find_closest_point_on_plane(points):
#     """
#     ３点を通る平面上にある最短距離の点の座標を求める関数

#     Parameters
#     ----------
#     points : numpy array, shape (3, 3)
#         ３点の３次元座標

#     Returns
#     -------
#     numpy array, shape (3,)
#         平面上にある最短距離の点の３次元座標
#     """
#     # 3点の座標を取得
#     point1 = points[0]
#     point2 = points[1]
#     point3 = points[2]

#     # 3点を通る平面の法線ベクトルを計算
#     normal = np.cross(point2 - point1, point3 - point1)
#     normal /= np.linalg.norm(normal)

#     # 平面の法線ベクトルに垂直な直線上にある最短距離の点を計算
#     # 3点の座標の平均を取り、法線ベクトルのスケール分だけ移動
#     center = np.mean(points, axis=0)
#     point_on_plane = center + normal

#     return point_on_plane

In [ ]:
# # 平面上にある最短距離の点の座標を求める
# closest_point_on_plane = find_closest_point_on_plane(points)
# # point_on_plane と point1 の距離を計算
# closest_point_distance = np.linalg.norm(closest_point_on_plane - points[0])
# print(closest_point_on_plane)
# print(closest_point_distance)

# sphere = o3d.geometry.TriangleMesh.create_sphere(radius=radius)
# sphere.translate(center)
# #point_cloud_sphere = generate_sphere_point_cloud(points)
# point_cloud_sphere = o3d.geometry.PointCloud()
# point_cloud_sphere.points = o3d.utility.Vector3dVector(np.asarray(sphere.vertices))
# point_cloud_sphere.paint_uniform_color([0, 1, 0])  # 赤色に設定


In [ ]:
# points2 = np.asarray(point_cloud_sphere.points)
# colors2 = np.asarray(point_cloud_sphere.colors)

# # 2つの点群データを連結
# points_combined = np.concatenate([np_z, points2], axis=0)
# points_combined_color = np.concatenate([np_z, colors2], axis=0)

# # 新しいPointCloudオブジェクトを作成
# point_cloud_combined = o3d.geometry.PointCloud()
# point_cloud_combined.points = o3d.utility.Vector3dVector(points_combined)
# point_cloud_combined.colors = o3d.utility.Vector3dVector(points_combined_color)

# # 融合された点群を表示
# o3d.visualization.draw_plotly([point_cloud_combined])

In [ ]:
##np_doki_down2 = np.asarray(pcd_doki_down2.points)

# 3列目で一番小さい値を取得
#min_value = np.max(np_doki_down2[:, 2])

#points_z0 = np_doki_down2[np_doki_down2[:, 2] == min_value]

#pcd_3 = o3d.geometry.PointCloud()
#pcd_3.points = o3d.utility.Vector3dVector(points_z0)

#print(points_z0)
#o3d.visualization.draw_plotly([pcd_3])

In [ ]:
# template = o3d.io.read_point_cloud('/content/drive/MyDrive/PointCloudData/point_cloud 5.ply')

# ## 現物とカラーチャートをダウンサンプリングして表示
# down_sample_pcd_plotly(template)
# colorbar_pcd_plotly(template)
# ## 座標とRGBを配列化
# np_temp = np.asarray(template.points)
# np_temp_col = np.asarray(template.colors) #* 255.0

# print(np_temp_col)

## xyzrgb　の並びにする
# df1 = pd.DataFrame(np_temp, columns=xyz)
# df2 = pd.DataFrame(np_temp_col, columns=rgb)
# df_pcd = pd.concat([df1, df2], axis=1)
# import cv2

# # RGB配列
# rgb = np_temp_col #np.array([[[255, 0, 0], [0, 255, 0], [0, 0, 255]]], dtype=np.uint8)
# rgb = rgb.astype(np.uint8)
# # BGR配列に変換（OpenCVではBGRを使用するため）
# bgr = cv2.cvtColor(rgb.reshape(-1, 1, 3), cv2.COLOR_RGB2BGR)

# # BGRからHSVに変換
# hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)

# print(hsv)
#print(hsv[0])

# pcd_hsv = o3d.geometry.PointCloud()
# pcd_hsv.points = o3d.utility.Vector3dVector(hsv)
# pcd_hsv.colors = o3d.utility.Vector3dVector(hsv)
# down_sample_pcd_plotly(pcd_hsv)

In [ ]:
# # 縮尺用の点を元に台座部分を識別


# # 変換行列の取得
# M = get_transform_matrix(scale_rows[xyz])

# # 変換行列を使って座標を変換する
# points = df_pcd[xyz].to_numpy()
# transformed_points = np.matmul(points, M)

# # 変換後の座標をデータフレームに格納
# df_transformed = pd.DataFrame(transformed_points, columns=['levelingx', 'levelingy', 'levelingz'])
# # print(df_transformed)
# df_pcd3 = pd.concat([df_pcd2, df_transformed], axis=1)
# df_pcd3['levelingz_border'] = df_pcd3[df_pcd2['type'] == 'green_scale']['z'].max()

# bottom_rows = df_pcd3.query('levelingz < levelingz_border')
# # print(bottom_rows[xyz])

# bottom_rows['type'] = 'bottom' 
# # df_pcd2.update(bottom_rows)
# print(bottom_rows[['levelingx', 'levelingy', 'levelingz']])
# separate_df_plotly(bottom_rows[['levelingx', 'levelingy', 'levelingz']],bottom_rows[rgb])

In [ ]:
# # 3次元空間の3点を示すNumpy配列を生成
# # points = np.array([[0.54242358, 0.94373941, 0.03549607],
# #                    [0.92990328, 0.85805119, 0.23877127],
# #                    [0.16066227, 0.62000717, 0.70497215]])

# points = kmeans.cluster_centers_

# # 2点間の距離を計算
# distances = pdist(points)

# # 距離行列を作成
# distance_matrix = squareform(distances)

# # 最も距離が大きい2点を探す
# i, j = np.unravel_index(np.argmax(distance_matrix), distance_matrix.shape)

# # 残りの1点のインデックスを取得
# k = np.setdiff1d(np.arange(3), [i, j])[0]

# # 距離を取得
# distances = distance_matrix[i, j]

# # 中心を計算
# #point_center,radius  = find_circle_center(points[0], points[1], points[2])
# # radius  = calculate_circle_radius(points)
# # 平面上にある最短距離の点の座標を求める
# #point_center = find_closest_point_on_plane(points)
# # point_on_plane と point1 の距離を計算
# #radius = np.linalg.norm(closest_point_on_plane - points[0])


# # 結果を出力
# point_i = points[i]
# point_j = points[j]
# point_k = points[k]

# print(f"最も距離が大きい2点の座標: {point_i}, {point_j}")
# print("距離:", distances)
# print(f"残りの1点の座標: {point_k}")
# #print(f"中心座標: {point_center}")
# #print(f"半径: {radius}")
# #print(point_center2,radius2)

In [ ]:


# def get_transform_matrix(df):
#     # データフレームの x, y, z 列を NumPy 配列に変換
#     points = df[['x', 'y', 'z']].to_numpy()
    
#     # 平面の法線ベクトルを計算
#     # ベクトル 1 = ベクトル 0 - ベクトル 5
#     v0 = points[0]
#     v5 = points[5]
#     n = v0 - v5
    
#     # 平面の回転を考慮した変換行列を計算
#     z = np.array([0, 0, 1])  # z 軸のベクトル
#     cos_theta = np.dot(n, z) / (np.linalg.norm(n) * np.linalg.norm(z))
#     sin_theta = np.sqrt(1 - cos_theta ** 2)
#     k = np.cross(z, n)
#     K = np.array([
#         [0, -k[2], k[1]],
#         [k[2], 0, -k[0]],
#         [-k[1], k[0], 0],
#     ])
#     R = np.identity(3) + sin_theta * K + (1 - cos_theta) * K @ K
#     T = np.array([[1, 0, 0], [0, 1, 0], [-v0[2], -v0[1], 1]])
#     M = T @ R
    
#     return M


In [ ]:
# 変換行列を使って座標を変換する
# points = scale_rows[xyz].to_numpy()
# transformed_points = np.matmul(points, M)
# df_transformed = pd.DataFrame(transformed_points, columns=xyz)
# print(transformed_points)
# separate_df_plotly(df_transformed,scale_rows[rgb])
